In [11]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# API endpoint for Binance order book
API_URL = "https://api.binance.com/api/v3/depth"

def fetch_order_book(symbol, limit=100):
    """Fetch the order book data from Binance API"""
    params = {'symbol': symbol, 'limit': limit}
    response = requests.get(API_URL, params=params)
    data = response.json()
    return data

def process_order_book(data):
    """Process order book data into pandas DataFrame"""
    bids = pd.DataFrame(data['bids'], columns=['Price', 'Quantity'], dtype=float)
    asks = pd.DataFrame(data['asks'], columns=['Price', 'Quantity'], dtype=float)
    return bids, asks

def calculate_order_book_metrics(bids, asks):
    """Calculate order book metrics"""
    spread = asks['Price'].iloc[0] - bids['Price'].iloc[0]
    bid_depth = bids['Quantity'].sum()
    ask_depth = asks['Quantity'].sum()
    liquidity_ratio = bid_depth / ask_depth
    return spread, bid_depth, ask_depth, liquidity_ratio

def prepare_features(bids, asks):
    """Prepare features for predictive modeling"""
    features = pd.DataFrame({
        'Bid_Price': bids['Price'],
        'Bid_Quantity': bids['Quantity'],
        'Ask_Price': asks['Price'],
        'Ask_Quantity': asks['Quantity']
    }).fillna(0)
    return features

def predict_price_movement(features):
    """Predict price movement using linear regression"""
    features['Mid_Price'] = (features['Bid_Price'] + features['Ask_Price']) / 2
    target = features['Mid_Price'].shift(-1) - features['Mid_Price']
    features = features.iloc[:-1]
    target = target.dropna()

    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error: {mse:.2f}")

    return model

# Example symbol
symbol = 'BTCUSDT'
order_book_data = fetch_order_book(symbol)
bids, asks = process_order_book(order_book_data)

# Calculate metrics
spread, bid_depth, ask_depth, liquidity_ratio = calculate_order_book_metrics(bids, asks)
print(f"Spread: {spread:.2f}")
print(f"Bid Depth: {bid_depth:.2f}")
print(f"Ask Depth: {ask_depth:.2f}")
print(f"Liquidity Ratio: {liquidity_ratio:.2f}")

# Prepare features and predict
features = prepare_features(bids, asks)
model = predict_price_movement(features)


Spread: 0.01
Bid Depth: 24.52
Ask Depth: 17.67
Liquidity Ratio: 1.39
Mean Squared Error: 0.03
